#0、引入依赖

In [1]:
import numpy as np
import pandas as pd
#直接引入sklearn数据集，iris鸢尾花
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split   #切分数据集为训练集和测试集
from sklearn.metrics import accuracy_score  #这里是用来计算分类预测的准确率

#1、数据加载和预处理

In [15]:
iris= load_iris()  #结果可以认为是一个对象
df = pd.DataFrame(data=iris.data,columns=iris.feature_names)
df['class']=iris.target
df['class']=df['class'].map({0:iris.target_names[0],1:iris.target_names[1],2:iris.target_names[2]})
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [20]:
#根据获取的数据取出 X ，Y
x = iris.data
y=iris.target.reshape(-1,1)  #使用reshape方法，把一行，转换成一列的向量数据

In [43]:
#划分训练集和测试集
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=35,stratify=y)
#####      按比例分割训练集和测试集，
print(x_train.shape,y_train.shape) #x训练集和y训练集
print(x_test.shape,y_test.shape)  #x测试集和y测试集


dist=np.array([1,54,84,15,48,64,6541,6181486,486541])
nn_index=np.argsort(dist)   ##argsort方法是对数组中数据大小，由小到大排序，返回下标值
print(nn_index)
nn_y=y_train[nn_index[:6]].ravel()  #ravel方法是将二维数组转换成一维数组，或者说向量数据
print(nn_y)
print(np.bincount(nn_y))   #bincount方法是统计一个向量数据中每个数出现的次数，并以下表数排列返回

(105, 4) (105, 1)
(45, 4) (45, 1)
[0 3 4 1 5 2 6 8 7]
[1 2 1 2 1 2]
[0 3 3]


#2、核心算法实现

In [30]:
#距离函数定义,曼哈顿公式，不开根号下的公式
def L1_distance(a,b):    #矩阵做减法的时候，只能是一个矩阵和一个向量
    return np.sum(np.abs(a-b),axis=1)  #轴是一，结果直接保存成一列   abs是绝对值
def L2_distance(a,b):
    return np.sqrt(np.sum((a-b)**2,axis=1))  #欧式距离的公式，sqrt是开根号

#分类器的实现 ，需要写面向对象函数
class  KNN(object):
    def __index__(self,n_neighbors=1,dis_func=L1_distance):
        self.n_neighbors=n_neighbors
        self.dis_func=dis_func
        
    #训练模型方法
    def fit(self,x,y):
        self.x_train=x_train
        self.y_train=y_train
    
    #模型预测方法
    def predict(self,x):  #对模型结果的预测，只需要传进去一个x，对x进行预测，不用传进y
        #初始化预测分类数据
        #创建一个都是零的，以x数组的行数，的以为数组保存结果，并对类型进行更改，改为整形
        y_pred=np.zeros((x.shape[0],1),dtype=self.y_train.dtype)
        #遍历输入的x数据点，取出每一个数据点的序号 i和对应的值
        for i, x_test in enumerate(x):
            #取出的输入点的值。跟所有训练数据计算距离
            distances=self.dis_func(self.x_train,x_test)
            
            #得到的循环按照由近到远排序，取出索引值
            nn_index=np.argsort(distances)
            
            #选取最近的k个点，保存他们对应的分类类型
            nn_y=self.y_train[nn_index[:self.n_neighbors]].ravel()
            
            #统计类别中出现频率最高的那个，赋值给y_pred[i]
            y_pred[i]=np.argmax(np.bincount(nn_y))
        return y_pred

#3、测试

In [34]:
#定义一个knn实例，
knn=KNN()
#训练模型
knn.fit(x_train,y_train)

#保存结果
result_list=[]

#针对不同的参数选取，做预测
for p in [1,2]:
    knn.dis_func=L1_distance if p ==1 else L2_distance
    for i in range(1,10,2):
        knn.n_neighbors=i
#传入测试数据，做预测，
        y_pred=knn.predict(x_test)
#求出预测准确率
        accuracy=accuracy_score(y_test,y_pred)
        result_list.append([i,'L1_distance' if p ==1 else 'L2_distance',accuracy])
resu=pd.DataFrame(result_list)
resu

,0,1,2
0,1,L1_distance,0.933333
1,3,L1_distance,0.933333
2,5,L1_distance,0.977778
3,7,L1_distance,0.955556
4,9,L1_distance,0.955556
5,1,L2_distance,0.933333
6,3,L2_distance,0.933333
7,5,L2_distance,0.977778
8,7,L2_distance,0.977778
9,9,L2_distance,0.977778
